## Introduction
This notebook runs all the processing steps one by one for several models and renders the output. Each section is individually runnable after a kernel restart 

## Observations
* MobileNetV3 small and EfficientNetV2 small display nicely
  * It is necessary to pan and zoom out and in a few times to really make sense of these networks
  * The interactive widget makes this much easier than the static figure
  * Adding the overview panel is also very valuable for these two models
* For CLIP and Yolov8 Nano, symbolic tracing via `torch.fx` fails due to symbolically traced variables being used as inputs to control flow
  * This prevents `idlmav` from displaying graphs for these models 

## MobileNetV3 small

In [1]:
import sys
sys.path.append('..')
import torch
from torchvision.models import mobilenet_v3_small
from idlmav import MAV, plotly_renderer

model = mobilenet_v3_small(weights=None)
model.eval()
inputs = torch.randn(1,3,224,224)
device = 'cpu'

mav = MAV(model, inputs, device)
with plotly_renderer('notebook_connected'): mav.show_figure()

/home/dev/ai/idlmav/.venv/lib/python3.10/site-packages/torchprofile/profile.py:22: UserWarning:

No handlers found: "aten::hardswish_". Skipped.

/home/dev/ai/idlmav/.venv/lib/python3.10/site-packages/torchprofile/profile.py:22: UserWarning:

No handlers found: "aten::hardsigmoid". Skipped.

/home/dev/ai/idlmav/.venv/lib/python3.10/site-packages/torchprofile/profile.py:22: UserWarning:

No handlers found: "aten::hardswish_". Skipped.

/home/dev/ai/idlmav/.venv/lib/python3.10/site-packages/torchprofile/profile.py:22: UserWarning:

No handlers found: "aten::hardswish_". Skipped.

/home/dev/ai/idlmav/.venv/lib/python3.10/site-packages/torchprofile/profile.py:22: UserWarning:

No handlers found: "aten::hardsigmoid". Skipped.

/home/dev/ai/idlmav/.venv/lib/python3.10/site-packages/torchprofile/profile.py:22: UserWarning:

No handlers found: "aten::hardswish_". Skipped.

/home/dev/ai/idlmav/.venv/lib/python3.10/site-packages/torchprofile/profile.py:22: UserWarning:

No handlers found: "aten:

In [2]:
mav.show_widget(add_overview=True)

## EfficientNetV2 small

In [3]:
import sys
sys.path.append('..')
import torch
from torchvision.models import efficientnet_v2_s
from idlmav import MAV, plotly_renderer

model = efficientnet_v2_s(pretrained=False)
model.eval()
inputs = torch.randn(1, 3, 224, 224)
device = 'cpu'

mav = MAV(model, inputs, device)
with plotly_renderer('notebook_connected'): mav.show_figure()

/home/dev/ai/idlmav/.venv/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning:

The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.

/home/dev/ai/idlmav/.venv/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning:

Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.

/home/dev/ai/idlmav/.venv/lib/python3.10/site-packages/torchprofile/profile.py:22: UserWarning:

No handlers found: "aten::silu_". Skipped.

/home/dev/ai/idlmav/.venv/lib/python3.10/site-packages/torchprofile/profile.py:22: UserWarning:

No handlers found: "aten::silu_". Skipped.

/home/dev/ai/idlmav/.venv/lib/python3.10/site-packages/torchprofile/profile.py:22: UserWarning:

No handlers found: "aten::silu_". Skipped.

/home/dev/ai/idlmav/.venv/lib/python3.10/site-packages/torchprofile/prof

In [4]:
mav.show_widget(add_overview=True)

## CLIP

In [5]:
import sys
sys.path.append('..')
from transformers import CLIPProcessor, CLIPModel
import torch
from idlmav import MAV, plotly_renderer

model = CLIPModel.from_pretrained("openai/clip-vit-base-patch16")
model.eval()
image_inputs = torch.randn(1, 3, 224, 224)
device = 'cpu'

# # Forward pass (only image encoder)
# output = model.vision_model(image_inputs)
# print("CLIP ViT output shape:", output.pooler_output.shape)

none_args = ['input_ids','attention_mask','position_ids','return_loss','output_attentions','output_hidden_states','interpolate_pos_encoding','return_dict']
concrete_args={arg_name:None for arg_name in none_args}
mav = MAV(model.vision_model, image_inputs, device, concrete_args=concrete_args)
with plotly_renderer('notebook_connected'): mav.show_figure()

TraceError: symbolically traced variables cannot be used as inputs to control flow

### `torch.compile` with custom `fx` backend
* Based on [This tutorial](https://pytorch.org/tutorials/intermediate/torch_compile_tutorial.html)

In [6]:
torch._dynamo.list_backends()

['cudagraphs', 'inductor', 'onnxrt', 'openxla', 'tvm']

In [8]:
def custom_backend(gm, example_inputs):
    print("")
    print("custom backend called with FX graph:")
    gm.graph.print_tabular()
    return gm.forward

torch._dynamo.reset()
compiled_model = torch.compile(model.vision_model.to(device), backend=custom_backend)
outputs = compiled_model(image_inputs)


custom backend called with FX graph:
opcode         name                                                                                                    target                                                                                                  args                                                                                                                                                                                                                            kwargs
-------------  ------------------------------------------------------------------------------------------------------  ------------------------------------------------------------------------------------------------------  ------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------  -----------------------------------------------------
placeho

## YOLOv11n (Nano)

In [2]:
import sys
sys.path.append('..')
from ultralytics import YOLO
import torch
from idlmav import MAV, plotly_renderer

model = YOLO("yolo11n.pt")
inputs = torch.rand(1, 3, 640, 640)
device = 'cpu'
model.predict(inputs)  # Causes internal model to be loaded and configured dynamically

concrete_args=dict(augment=False, visualize=False, embed=None)
mav = MAV(model.predictor.model, inputs, device, concrete_args=concrete_args)
with plotly_renderer('notebook_connected'): mav.show_figure()


0: 640x640 (no detections), 33.4ms
Speed: 1.5ms preprocess, 33.4ms inference, 7.6ms postprocess per image at shape (1, 3, 640, 640)


TraceError: Proxy object cannot be iterated. This can be attempted when the Proxy is used in a loop or as a *args or **kwargs function argument. See the torch.fx docs on pytorch.org for a more detailed explanation of what types of control flow can be traced, and check out the Proxy docstring for help troubleshooting Proxy iteration errors

### `torch.compile` with custom `fx` backend
* Based on [This tutorial](https://pytorch.org/tutorials/intermediate/torch_compile_tutorial.html)

In [5]:
def custom_backend(gm, example_inputs):
    print("")
    print("custom backend called with FX graph:")
    gm.graph.print_tabular()
    return gm.forward

torch._dynamo.reset()
compiled_model = torch.compile(model.to(device), backend=custom_backend)
outputs = compiled_model(inputs)


custom backend called with FX graph:
opcode       name                                       target                                     args                                                                          kwargs
-----------  -----------------------------------------  -----------------------------------------  ----------------------------------------------------------------------------  --------
placeholder  l_stack0_modules_model_modules_23_stride   L_stack0_modules_model_modules_23_stride   ()                                                                            {}
placeholder  l_stack0_modules_model_modules_23_anchors  L_stack0_modules_model_modules_23_anchors  ()                                                                            {}
placeholder  l_stack0_modules_model_modules_23_strides  L_stack0_modules_model_modules_23_strides  ()                                                                            {}
call_method  to                                     

/home/dev/ai/idlmav/.venv/lib/python3.10/site-packages/torch/_dynamo/variables/functions.py:725: UserWarning:

Graph break due to unsupported builtin time.time. This function is either a Python builtin (e.g. _warnings.warn) or a third-party C/C++ Python extension (perhaps created with pybind). If it is a Python builtin, please file an issue on GitHub so the PyTorch team can add support for it and see the next case for a workaround. If it is a third-party C/C++ Python extension, please either wrap it into a PyTorch-understood custom operator (see https://pytorch.org/tutorials/advanced/custom_ops_landing_page.html for more details) or, if it is traceable, use torch.compiler.allow_in_graph.




custom backend called with FX graph:
opcode         name                                                                                                                             target                                                                                                                           args                                                                                                                                                                                                                                                                                                  kwargs
-------------  -------------------------------------------------------------------------------------------------------------------------------  -------------------------------------------------------------------------------------------------------------------------------  -------------------------------------------------------------------------------------------------------------------

/home/dev/ai/idlmav/.venv/lib/python3.10/site-packages/torch/_dynamo/variables/functions.py:725: UserWarning:

Graph break due to unsupported builtin sys.intern. This function is either a Python builtin (e.g. _warnings.warn) or a third-party C/C++ Python extension (perhaps created with pybind). If it is a Python builtin, please file an issue on GitHub so the PyTorch team can add support for it and see the next case for a workaround. If it is a third-party C/C++ Python extension, please either wrap it into a PyTorch-understood custom operator (see https://pytorch.org/tutorials/advanced/custom_ops_landing_page.html for more details) or, if it is traceable, use torch.compiler.allow_in_graph.

